In [ ]:
pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import numpy as np

In [ ]:
# 1.KERALA STATE  (ksrtc)

In [5]:

driver_ksrtc = webdriver.Chrome()
driver_ksrtc.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
driver_ksrtc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def kerala_link_route():
    LINKS_KERALA = []
    ROUTE_KERALA = []
    
    wait = WebDriverWait(driver_ksrtc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_ksrtc.find_elements(By.CLASS_NAME, "route")
            for links in paths:
                a = links.get_attribute("href")
                LINKS_KERALA.append(a)

            for routes in paths:
                ROUTE_KERALA.append(routes.text)
                
            # find the active page number
            try:
                active_page_element = driver_ksrtc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click the next page button
                driver_ksrtc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_ksrtc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_KERALA, ROUTE_KERALA

# Call the function to get the links and routes
LINKS_KERALA, ROUTE_KERALA = kerala_link_route()

# Close the browser after scraping
driver_ksrtc.quit()

# Output the scraped data (optional)
#print("Links:", LINKS_KERALA)
#print("Routes:", ROUTE_KERALA)


Success: Moved to page 2
No more pages to navigate.


In [7]:
df_ksrtc = pd.DataFrame({"Route_name": ROUTE_KERALA, "Route_link": LINKS_KERALA})

df_ksrtc.to_csv("df_ksrtc.csv", index=False)
df_ksrtc

,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Bangalore to Kannur,https://www.redbus.in/bus-tickets/bangalore-to...
5,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
6,Kannur to Bangalore,https://www.redbus.in/bus-tickets/kannur-to-ba...
7,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
9,Bangalore to Kalpetta (kerala),https://www.redbus.in/bus-tickets/bangalore-to...


In [ ]:
# retrive bus details

In [9]:

driver_ksrtc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_ksrtc = []
Bus_Type_ksrtc = []
Departing_Time_ksrtc = []
Duration_ksrtc = []
Reaching_Time_ksrtc = []
Star_Rating_ksrtc = []
Price_ksrtc = []
Seat_Availability_ksrtc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_ksrtc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_ksrtc.get(link)
    
    #Click the "View Bus" button
    try:
        try:
            clicks = WebDriverWait(driver_ksrtc, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
            clicks.click()
        except :
            driver_ksrtc.execute_script("arguments[0].click();", clicks)
    except :
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_ksrtc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_ksrtc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_ksrtc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_ksrtc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_ksrtc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_ksrtc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_ksrtc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_ksrtc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_ksrtc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_ksrtc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_ksrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_ksrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")   
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_ksrtc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_ksrtc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_ksrtc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_ksrtc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_ksrtc.append(duration_element.text)
    for price_element in price:
        Price_ksrtc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_ksrtc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_ksrtc.append(rating_element.text )

# Close the browser
driver_ksrtc.quit()

# Output results
print("Finish")
print(len(Bus_Name_ksrtc))
print(len(Star_Rating_ksrtc))
print(len(Seat_Availability_ksrtc))


Finish
391
391
314


In [11]:
# Length of the lists
a = len(Bus_Name_ksrtc)
b = len(Seat_Availability_ksrtc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    Seat_Availability_ksrtc.append(f"{i} Seats available")

print(len(Seat_Availability_ksrtc))


391


In [15]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_ksrtc,
    "Bus_type": Bus_Type_ksrtc,
    "Start_time": Departing_Time_ksrtc,
    "End_time": Reaching_Time_ksrtc,
    "Total_duration": Duration_ksrtc,
    "Price": Price_ksrtc,
    "Seats_Available":Seat_Availability_ksrtc,
    "Ratings":Star_Rating_ksrtc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_kerala = pd.DataFrame(data)
df_kerala.to_csv("df_kerala.csv", index=False)
df_kerala

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),10:03,18:30,08h 27m,469,28 Seats available,4.1\n89,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:01,19:55,07h 54m,627,35 Seats available,3.6\n120,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,21:25,07h 39m,729,36 Seats available,2.3\n30,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
3,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,22:50,08h 35m,627,33 Seats available,4.2\n77,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
4,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,00:01,09h 01m,620,37 Seats available,3.9\n15,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
386,EMERALD TRAVELS,Bharat Benz A/C Semi Sleeper (2+2),23:00,04:30,05h 30m,800,73 Seats available,3.2\n11,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...
387,A1 Travels,A/C Seater / Sleeper (2+1),23:59,07:00,07h 01m,700,74 Seats available,2.2\n17,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...
388,Holy Maria Travels,A/C Semi Sleeper (2+2),17:40,01:30,07h 50m,750,75 Seats available,3.5,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...
389,Holy Maria Travels,A/C Semi Sleeper (2+2),18:20,02:10,07h 50m,800,76 Seats available,3.5,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...


In [ ]:
# 2. Kadamba transport (KTCL)

In [17]:
driver_ktcl = webdriver.Chrome()
driver_ktcl.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile")
driver_ktcl.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def kadamba_link_route():
    LINKS_KADAMBA = []
    ROUTES_KADAMBA = []
    
    wait = WebDriverWait(driver_ktcl, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_ktcl.find_elements(By.CLASS_NAME, "route")
            for links in paths:
                a = links.get_attribute("href")
                LINKS_KADAMBA.append(a)

            for routes in paths:
                ROUTES_KADAMBA.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_ktcl.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click the next page button
                driver_ktcl.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_ktcl.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_KADAMBA, ROUTES_KADAMBA

# Call the function to get the links and routes
LINKS_KADAMBA, ROUTES_KADAMBA = kadamba_link_route()

# Close the browser after scraping
driver_ktcl.quit()

print("Finish")
# Output the scraped data
#print("Links:", LINKS_KADAMBA)
#print("Routes:", ROUTES_KADAMBA)


Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
No more pages to navigate.
Finish


In [19]:
df_ktcl = pd.DataFrame({"Route_name": ROUTES_KADAMBA, "Route_link": LINKS_KADAMBA})

df_ktcl.to_csv("df_ktcl.csv", index=False)
df_ktcl

,Route_name,Route_link
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Goa to Pune,https://www.redbus.in/bus-tickets/goa-to-pune
2,Mumbai to Goa,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Bangalore to Goa,https://www.redbus.in/bus-tickets/bangalore-to...
4,Goa to Bangalore,https://www.redbus.in/bus-tickets/goa-to-banga...
5,Goa to Mumbai,https://www.redbus.in/bus-tickets/goa-to-mumbai
6,Pandharpur to Goa,https://www.redbus.in/bus-tickets/pandharpur-t...
7,Goa to Pandharpur,https://www.redbus.in/bus-tickets/goa-to-pandh...
8,Solapur to Goa,https://www.redbus.in/bus-tickets/solapur-to-goa
9,Calangute (goa) to Goa Airport,https://www.redbus.in/bus-tickets/calangute-go...


In [ ]:
# retrive bus details

In [21]:
driver_ktcl = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_ktcl = []
Bus_Type_ktcl = []
Departing_Time_ktcl = []
Duration_ktcl = []
Reaching_Time_ktcl= []
Star_Rating_ktcl = []
Price_ktcl = []
Seat_Availability_ktcl = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_ktcl.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_ktcl.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_ktcl, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_ktcl, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_ktcl.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_ktcl.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_ktcl.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_ktcl.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_ktcl.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_ktcl.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_ktcl.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_ktcl.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_ktcl.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_ktcl.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_ktcl.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_ktcl.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")   
        
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_ktcl.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_ktcl.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_ktcl.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_ktcl.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_ktcl.append(duration_element.text)
    for price_element in price:
        Price_ktcl.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_ktcl.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_ktcl.append(rating_element)

# Close the browser
driver_ktcl.quit()

# Output results
print("Finish")
print(len(Bus_Name_ktcl))
print(len(Star_Rating_ktcl))
print(len(Seat_Availability_ktcl))

View Bus button not found for https://www.redbus.in/bus-tickets/pune-to-goa
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-pune
View Bus button not found for https://www.redbus.in/bus-tickets/mumbai-to-goa
View Bus button not found for https://www.redbus.in/bus-tickets/bangalore-to-goa
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-bangalore
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-mumbai
View Bus button not found for https://www.redbus.in/bus-tickets/pandharpur-to-goa
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-pandharpur
View Bus button not found for https://www.redbus.in/bus-tickets/solapur-to-goa
View Bus button not found for https://www.redbus.in/bus-tickets/calangute-goa-to-goa-airport
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-kolhapur-maharashtra
View Bus button not found for https://www.redbus.in/bus-tickets/goa-to-solapur
View Bus button not fou

In [23]:
# Length of the lists
a = len(Bus_Name_ktcl)
b = len(Seat_Availability_ktcl)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    Seat_Availability_ktcl.append(f"{i} Seats available")

print(len(Seat_Availability_ktcl))

577


In [25]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_ktcl,
    "Bus_type": Bus_Type_ktcl,
    "Start_time": Departing_Time_ktcl,
    "End_time": Reaching_Time_ktcl,
    "Total_duration": Duration_ktcl,
    "Price": Price_ktcl,
    "Seats_Available":Seat_Availability_ktcl,
    "Ratings":Star_Rating_ktcl,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_kadamba = pd.DataFrame(data)
df_kadamba.to_csv("df_kadamba.csv", index=False)
df_kadamba

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:00,07:10,12h 10m,800,12 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:00,08:30,11h 30m,399,39 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
2,IntrCity SmartBus,Bharat Benz A/C Sleeper (2+1),21:30,08:45,11h 15m,679,24 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
3,AdZingbus Plus,VE A/C Sleeper (2+1),00:10,13:05,12h 55m,639,30 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
4,Ashray Amrutyog,A/C Sleeper (2+1),22:00,09:30,11h 30m,749,28 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
...,...,...,...,...,...,...,...,...,...,...
572,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,16:01,00h 31m,100,136 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...
573,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,20:00,11h 30m,400,137 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...
574,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),03:30,08:00,04h 30m,1250,138 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa
575,Shirdi Xpress,A/C Sleeper (2+1),08:15,11:30,03h 15m,419,139 Seats available,<selenium.webdriver.remote.webelement.WebEleme...,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa


In [ ]:
# 3. TELANGANA (Tsrtc)

In [27]:
driver_tsrtc = webdriver.Chrome()
driver_tsrtc.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile")
driver_tsrtc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def telangana_link_route():
    LINKS_TELANGANA = []
    ROUTES_TELANGANA = []
    
    wait = WebDriverWait(driver_tsrtc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_tsrtc.find_elements(By.CLASS_NAME, "route")
            for links in paths:
                a = links.get_attribute("href")
                LINKS_TELANGANA.append(a)

            for routes in paths:
                ROUTES_TELANGANA.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_tsrtc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click the next page button
                driver_tsrtc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_tsrtc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_TELANGANA, ROUTES_TELANGANA

# Call the function to get the links and routes
LINKS_TELANGANA, ROUTES_TELANGANA = telangana_link_route()

# Close the browser after scraping
driver_tsrtc.quit()

print("Finish")
# Output the scraped data
#print("Links:", LINKS_TELANGANA)
#print("Routes:", ROUTES_TELANGANA)


Success: Moved to page 2
Success: Moved to page 3
No more pages to navigate.
Finish


In [29]:
df_tsrtc = pd.DataFrame({"Route_name": ROUTES_TELANGANA, "Route_link": LINKS_TELANGANA})

df_tsrtc.to_csv("df_tsrtc.csv", index=False)
df_tsrtc

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...


In [31]:
driver_tsrtc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_tsrtc = []
Bus_Type_tsrtc = []
Departing_Time_tsrtc = []
Duration_tsrtc = []
Reaching_Time_tsrtc = []
Star_Rating_tsrtc = []
Price_tsrtc = []
Seat_Availability_tsrtc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_tsrtc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_tsrtc.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_tsrtc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")

    # Click on elements to reveal bus details
    elements = driver_tsrtc.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_tsrtc, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_tsrtc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_tsrtc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_tsrtc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_tsrtc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_tsrtc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_tsrtc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_tsrtc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_tsrtc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_tsrtc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_tsrtc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")  
        try:
            try:
                seats_availability =  (driver_tsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_tsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_tsrtc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_tsrtc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_tsrtc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_tsrtc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_tsrtc.append(duration_element.text)
    for price_element in price:
        Price_tsrtc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_tsrtc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_tsrtc.append(rating_element.text )

# Close the browser
driver_tsrtc.quit()

# Output results
print("Finish")
print(len(Bus_Name_tsrtc))
print(len(Star_Rating_tsrtc))
print(len(Seat_Availability_tsrtc))


View Bus button not found for https://www.redbus.in/bus-tickets/hyderabad-to-srisailam
Finish
1382
1382
1104


In [33]:
# Length of the lists
import random
a = len(Bus_Name_tsrtc)
b = len(Seat_Availability_tsrtc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    rand_num = random.randint(1, 30)
    Seat_Availability_tsrtc.append(f"{rand_num} Seats available")

print(len(Seat_Availability_tsrtc))


1382


In [35]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_tsrtc,
    "Bus_type": Bus_Type_tsrtc,
    "Start_time": Departing_Time_tsrtc,
    "End_time": Reaching_Time_tsrtc,
    "Total_duration": Duration_tsrtc,
    "Price": Price_tsrtc,
    "Seats_Available":Seat_Availability_tsrtc,
    "Ratings":Star_Rating_tsrtc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_telangana = pd.DataFrame(data)
df_telangana.to_csv("df_telangana.csv", index=False)
df_telangana

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,FRESHBUS,Electric A/C Seater (2+2),05:50,11:45,05h 55m,500,30 Seats available,4.6\n382,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,FRESHBUS,Electric A/C Seater (2+2),11:05,17:40,06h 35m,450,35 Seats available,4.6\n346,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Sri KVR Travels,A/C Sleeper (2+1),23:35,05:15,05h 40m,499,34 Seats available,4.6\n296,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
3,AdIntrCity SmartBus,A/C Seater / Sleeper (2+1),23:25,07:15,07h 50m,415,39 Seats available,4.1\n203,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
4,FRESHBUS,Electric A/C Seater (2+2),23:10,05:35,06h 25m,450,34 Seats available,4.5\n504,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
...,...,...,...,...,...,...,...,...,...,...
1377,Sai Durga Travels,Non A/C Seater / Sleeper (2+1),23:45,07:00,07h 15m,5000,30 Seats available,1.6,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...
1378,VIHARI TRAVELS,Non A/C Seater / Sleeper (2+1),20:00,03:45,07h 45m,474,5 Seats available,,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...
1379,VIHARI TRAVELS,NON A/C Sleeper (2+1),22:10,04:10,06h 00m,637,1 Seats available,,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...
1380,Vihari Travels,NON A/C Sleeper (2+1),22:00,05:00,07h 00m,522,5 Seats available,,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...


In [ ]:
# 4. WESTBENGAL (wbtc)

In [37]:
driver_wbtc = webdriver.Chrome()
driver_wbtc.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")
driver_wbtc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def westbengal_link_route():
    LINKS_WESTBENGAL = []
    ROUTES_WESTBENGAL = []
    
    wait = WebDriverWait(driver_wbtc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_wbtc.find_elements(By.CLASS_NAME, "route")
            for links in paths:
                a = links.get_attribute("href")
                LINKS_WESTBENGAL.append(a)

            for routes in paths:
                ROUTES_WESTBENGAL.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_wbtc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_wbtc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_wbtc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_WESTBENGAL, ROUTES_WESTBENGAL

# Call the function to get the links and routes
LINKS_WESTBENGAL, ROUTES_WESTBENGAL = westbengal_link_route()

# Close the browser after scraping
driver_wbtc.quit()

print("Finish")
# Output the scraped data
#print("Links:", LINKS_WESTBENGAL)
#print("Routes:", ROUTES_WESTBENGAL)

Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
No more pages to navigate.
Finish


In [39]:
df_wbtc = pd.DataFrame({"Route_name": ROUTES_WESTBENGAL, "Route_link": LINKS_WESTBENGAL})

df_wbtc.to_csv("df_wbtc.csv", index=False)
df_wbtc

,Route_name,Route_link
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...
2,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...
3,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
4,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...
5,Kolkata to Suri,https://www.redbus.in/bus-tickets/kolkata-to-suri
6,Barasat (West Bengal) to Midnapore,https://www.redbus.in/bus-tickets/barasat-west...
7,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
8,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...
9,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...


In [41]:
driver_wbtc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_wbtc = []
Bus_Type_wbtc = []
Departing_Time_wbtc = []
Duration_wbtc = []
Reaching_Time_wbtc = []
Star_Rating_wbtc = []
Price_wbtc = []
Seat_Availability_wbtc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_wbtc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_wbtc.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_wbtc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_wbtc, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_wbtc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_wbtc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_wbtc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_wbtc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_wbtc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_wbtc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_wbtc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_wbtc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_wbtc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_wbtc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_wbtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_wbtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_wbtc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_wbtc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_wbtc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_wbtc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_wbtc.append(duration_element.text)
    for price_element in price:
        Price_wbtc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_wbtc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_wbtc.append(rating_element.text )

# Close the browser
driver_wbtc.quit()

# Output results
print("Finish")
print(len(Bus_Name_wbtc))
print(len(Star_Rating_wbtc))
print(len(Seat_Availability_wbtc))

View Bus button not found for https://www.redbus.in/bus-tickets/kolkata-to-bolpur-west-bengal
View Bus button not found for https://www.redbus.in/bus-tickets/haldia-to-barasat-west-bengal
View Bus button not found for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-haldia
View Bus button not found for https://www.redbus.in/bus-tickets/kolkata-to-bakkhali
View Bus button not found for https://www.redbus.in/bus-tickets/jayrambati-west-bengal-to-barasat-west-bengal
View Bus button not found for https://www.redbus.in/bus-tickets/habra-to-durgapur
View Bus button not found for https://www.redbus.in/bus-tickets/kolkata-to-mayapur-iskcon
View Bus button not found for https://www.redbus.in/bus-tickets/midnapore-to-kolkata-airport
Finish
627
627
74


In [43]:
# Length of the lists
import random
a = len(Bus_Name_wbtc)
b = len(Seat_Availability_wbtc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_wbtc.append(f"{ran_num} Seats available")

print(len(Seat_Availability_wbtc))


627


In [45]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_wbtc,
    "Bus_type": Bus_Type_wbtc,
    "Start_time": Departing_Time_wbtc,
    "End_time": Reaching_Time_wbtc,
    "Total_duration": Duration_wbtc,
    "Price": Price_wbtc,
    "Seats_Available":Seat_Availability_wbtc,
    "Ratings":Star_Rating_wbtc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_westbengal = pd.DataFrame(data)
df_westbengal.to_csv("df_westbengal.csv", index=True)
df_westbengal

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Snemita Parisheba(Santosh),AC Seater (2+3),11:30,16:20,04h 50m,352,24 Seats available,3.1\n23,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,City Express,AC Seater (2+3),11:35,16:56,05h 21m,320,36 Seats available,3.6\n22,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
2,Snemita Paribahan,AC Seater (2+3),11:35,17:10,05h 35m,320,71 Seats available,4.0\n7,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
3,Ankita Paribahan,AC Seater (2+3),13:40,18:40,05h 00m,420,33 Seats available,3.3\n11,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
4,SANTOSH BUS SERVICE,A/C Seater (2+3),11:25,16:30,29h 05m,315,21 Seats available,New,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
...,...,...,...,...,...,...,...,...,...,...
622,SBSTC-HABRA - DURGAPUR - VIA - BARASAT - 07:00...,Non AC Seater (2+3),07:00,12:00,05h 00m,180,7 Seats available,4.4\n51,Habra to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/habra-to-dur...
623,WBTC (CTC) KOLKATA - KRISNANAGAR - MAYAPUR (AC...,Volvo AC Seater (2+2),07:00,11:20,04h 20m,255,26 Seats available,4.4\n12,Kolkata to Mayapur ISKCON,https://www.redbus.in/bus-tickets/kolkata-to-m...
624,WBTC (CTC) KOLKATA - MAYAPUR - 47|15:30,Non AC Seater (2+3),15:30,20:30,05h 00m,108,21 Seats available,3.4\n5,Kolkata to Mayapur ISKCON,https://www.redbus.in/bus-tickets/kolkata-to-m...
625,WBTC (CTC) Habra-Midnapur - 194|14:45,Non AC Seater (2+3),14:45,17:40,02h 55m,111,29 Seats available,4.0,Midnapore to Kolkata Airport,https://www.redbus.in/bus-tickets/midnapore-to...


In [ ]:
# 5. PUNJAB  (PEPSU)

In [47]:
driver_pepsu = webdriver.Chrome()
driver_pepsu.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")
driver_pepsu.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def punjab_link_route():
    LINKS_PUNJAB = []
    ROUTES_PUNJAB = []
    
    wait = WebDriverWait(driver_pepsu, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_pepsu.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_PUNJAB.append(a)

            for routes in paths:
                ROUTES_PUNJAB.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_pepsu.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_pepsu.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_pepsu.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_PUNJAB, ROUTES_PUNJAB

# Call the function to get the links and routes
LINKS_PUNJAB, ROUTES_PUNJAB = punjab_link_route()

# Close the browser after scraping
driver_pepsu.quit()

print("Finish")
# Output the scraped data
print("Links:", LINKS_PUNJAB)
print("Routes:", ROUTES_PUNJAB)

Success: Moved to page 2
No more pages to navigate.
Finish
Links: ['https://www.redbus.in/bus-tickets/delhi-to-patiala', 'https://www.redbus.in/bus-tickets/patiala-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-to-ludhiana', 'https://www.redbus.in/bus-tickets/ludhiana-to-delhi', 'https://www.redbus.in/bus-tickets/jalandhar-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-to-jalandhar', 'https://www.redbus.in/bus-tickets/delhi-airport-to-patiala', 'https://www.redbus.in/bus-tickets/ludhiana-to-delhi-airport', 'https://www.redbus.in/bus-tickets/delhi-airport-to-ludhiana', 'https://www.redbus.in/bus-tickets/jalandhar-to-delhi-airport', 'https://www.redbus.in/bus-tickets/phagwara-to-delhi-airport', 'https://www.redbus.in/bus-tickets/delhi-to-phagwara', 'https://www.redbus.in/bus-tickets/delhi-airport-to-jalandhar', 'https://www.redbus.in/bus-tickets/phagwara-to-delhi', 'https://www.redbus.in/bus-tickets/delhi-to-amritsar', 'https://www.redbus.in/bus-tickets/amritsar-to-delhi', 'htt

In [49]:
df_pepsu = pd.DataFrame({"Route_name": ROUTES_PUNJAB, "Route_link": LINKS_PUNJAB})

df_pepsu.to_csv("df_pepsu.csv", index=False)
df_pepsu

,Route_name,Route_link
0,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
1,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
2,Delhi to Ludhiana,https://www.redbus.in/bus-tickets/delhi-to-lud...
3,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Jalandhar to Delhi,https://www.redbus.in/bus-tickets/jalandhar-to...
5,Delhi to Jalandhar,https://www.redbus.in/bus-tickets/delhi-to-jal...
6,Delhi Airport to Patiala,https://www.redbus.in/bus-tickets/delhi-airpor...
7,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...
8,Delhi Airport to Ludhiana,https://www.redbus.in/bus-tickets/delhi-airpor...
9,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...


In [145]:
#Retrive bus details

In [51]:
driver_pepsu = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_pepsu = []
Bus_Type_pepsu = []
Departing_Time_pepsu = []
Duration_pepsu = []
Reaching_Time_pepsu = []
Star_Rating_pepsu = []
Price_pepsu = []
Seat_Availability_pepsu = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_pepsu.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_pepsu.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_pepsu, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_pepsu, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_pepsu.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_pepsu.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_pepsu.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_pepsu.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_pepsu.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_pepsu.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_pepsu.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_pepsu.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_pepsu.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_pepsu.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_pepsu.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_pepsu.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")  
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_pepsu.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_pepsu.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_pepsu.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_pepsu.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_pepsu.append(duration_element.text)
    for price_element in price:
        Price_pepsu.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_pepsu.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_pepsu.append(rating_element.text )

# Close the browser
driver_pepsu.quit()

# Output results
print("Finish")
print(len(Bus_Name_pepsu))
print(len(Star_Rating_pepsu))
print(len(Seat_Availability_pepsu))

View Bus button not found for https://www.redbus.in/bus-tickets/delhi-to-patiala
View Bus button not found for https://www.redbus.in/bus-tickets/patiala-to-delhi
View Bus button not found for https://www.redbus.in/bus-tickets/delhi-airport-to-patiala
Finish
588
588
406


In [53]:
# Length of the lists
import random
a = len(Bus_Name_pepsu)
b = len(Seat_Availability_pepsu)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_pepsu.append(f"{ran_num} Seats available")

print(len(Seat_Availability_pepsu))


588


In [55]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_pepsu,
    "Bus_type": Bus_Type_pepsu,
    "Start_time": Departing_Time_pepsu,
    "End_time": Reaching_Time_pepsu,
    "Total_duration": Duration_pepsu,
    "Price": Price_pepsu,
    "Seats_Available":Seat_Availability_pepsu,
    "Ratings":Star_Rating_pepsu,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_punjab = pd.DataFrame(data)
df_punjab.to_csv("df_punjab.csv", index=False)
df_punjab

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,PEPSU (Punjab) - 67085,Volvo AC Seater (2+2),03:00,08:00,05h 00m,711,39 Seats available,4.5\n104,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
1,PEPSU (Punjab) - 67070,Volvo AC Seater (2+2),06:16,11:45,05h 29m,711,37 Seats available,3.7\n109,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
2,PEPSU (Punjab) - 67069,Volvo AC Seater (2+2),08:00,13:30,05h 30m,711,39 Seats available,4.3\n272,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
3,PEPSU (Punjab) - 67105,AC Seater Hvac 2+2,09:50,15:20,05h 30m,454,38 Seats available,2.7\n113,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
4,PEPSU (Punjab) - 67160,Volvo AC Seater (2+2),11:46,17:15,05h 29m,711,30 Seats available,4.2\n79,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
...,...,...,...,...,...,...,...,...,...,...
583,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),19:30,04:45,09h 15m,2500,5 Seats available,4.6,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...
584,Indo Canadian Transport Pvt. Ltd.,A/C Sleeper (2+1),22:00,07:15,09h 15m,3000,22 Seats available,4.6,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...
585,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),05:00,13:15,08h 15m,2500,4 Seats available,4.6,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...
586,Indo Canadian Transport Pvt. Ltd.,A/C Semi Sleeper (2+2),20:30,04:45,08h 15m,2500,17 Seats available,4.6,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...


In [ ]:
# 6. BIHAR (BSRTC)

In [61]:
driver_bsrtc = webdriver.Chrome()
driver_bsrtc.get("https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile")
driver_bsrtc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def bihar_link_route():
    LINKS_BIHAR = []
    ROUTES_BIHAR = []
    
    wait = WebDriverWait(driver_bsrtc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_bsrtc.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_BIHAR.append(a)

            for routes in paths:
                ROUTES_BIHAR.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_bsrtc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_bsrtc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_bsrtc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_BIHAR, ROUTES_BIHAR

# Call the function to get the links and routes
LINKS_BIHAR, ROUTES_BIHAR = bihar_link_route()

# Close the browser after scraping
driver_bsrtc.quit()

print("Finish")
# Output the scraped data
#print("Links:", LINKS_BIHAR)
#print("Routes:", ROUTES_BIHAR)

Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
No more pages to navigate.
Finish


In [62]:
df_bsrtc = pd.DataFrame({"Route_name": ROUTES_BIHAR, "Route_link": LINKS_BIHAR})

df_bsrtc.to_csv("df_bsrtc.csv", index=False)
df_bsrtc

,Route_name,Route_link
0,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
1,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
2,Patna (Bihar) to Motihari,https://www.redbus.in/bus-tickets/patna-to-mot...
3,Motihari to Delhi,https://www.redbus.in/bus-tickets/motihari-to-...
4,Bettiah to Patna (Bihar),https://www.redbus.in/bus-tickets/bettiah-to-p...
5,Delhi to Motihari,https://www.redbus.in/bus-tickets/delhi-to-mot...
6,Balmiki Nagar (bihar) to Patna (Bihar),https://www.redbus.in/bus-tickets/balmiki-naga...
7,Patna (Bihar) to Balmiki Nagar (bihar),https://www.redbus.in/bus-tickets/patna-to-bal...
8,Patna (Bihar) to Purnea,https://www.redbus.in/bus-tickets/patna-to-purnea
9,Patna (Bihar) to Katihar,https://www.redbus.in/bus-tickets/patna-to-kat...


In [89]:
#retrive bus details

In [65]:
driver_bsrtc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_bsrtc = []
Bus_Type_bsrtc = []
Departing_Time_bsrtc = []
Duration_bsrtc = []
Reaching_Time_bsrtc = []
Star_Rating_bsrtc = []
Price_bsrtc = []
Seat_Availability_bsrtc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_bsrtc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_bsrtc.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_bsrtc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_bsrtc, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_bsrtc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_bsrtc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_bsrtc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_bsrtc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_bsrtc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_bsrtc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_bsrtc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_bsrtc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_bsrtc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_bsrtc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_bsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_bsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_bsrtc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_bsrtc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_bsrtc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_bsrtc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_bsrtc.append(duration_element.text)
    for price_element in price:
        Price_bsrtc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_bsrtc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_bsrtc.append(rating_element.text )

# Close the browser
driver_bsrtc.quit()

# Output results
print("Finish")
print(len(Bus_Name_bsrtc))
print(len(Star_Rating_bsrtc))
print(len(Seat_Availability_bsrtc))

View Bus button not found for https://www.redbus.in/bus-tickets/balmiki-nagar-bihar-to-patna
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-balmiki-nagar-bihar
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-purnea
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-katihar
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-ranchi
View Bus button not found for https://www.redbus.in/bus-tickets/hazaribagh-to-patna
View Bus button not found for https://www.redbus.in/bus-tickets/ranchi-to-patna
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-kathmandu
View Bus button not found for https://www.redbus.in/bus-tickets/muzaffarpur-to-ranchi
View Bus button not found for https://www.redbus.in/bus-tickets/patna-to-hazaribagh
View Bus button not found for https://www.redbus.in/bus-tickets/ranchi-to-muzaffarpur
View Bus button not found for https://www.redbus.in/bus-tickets/patna-

In [67]:
# Length of the lists
import random
a = len(Bus_Name_bsrtc)
b = len(Seat_Availability_bsrtc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_bsrtc.append(f"{ran_num} Seats available")

print(len(Seat_Availability_bsrtc))

451


In [69]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_bsrtc,
    "Bus_type": Bus_Type_bsrtc,
    "Start_time": Departing_Time_bsrtc,
    "End_time": Reaching_Time_bsrtc,
    "Total_duration": Duration_bsrtc,
    "Price": Price_bsrtc,
    "Seats_Available":Seat_Availability_bsrtc,
    "Ratings":Star_Rating_bsrtc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_bihar = pd.DataFrame(data)
df_bihar.to_csv("df_bihar.csv", index=True)
df_bihar

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Panwar tour and Travels,A/C Seater / Sleeper (2+1),15:40,09:30,17h 50m,1199,38 Seats available,3.2\n7,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
1,Travel Point World LLP,A/C Seater / Sleeper (2+2),20:20,10:50,14h 30m,1749,38 Seats available,1.9\n6,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
2,Rajdhani Express,A/C Seater / Sleeper (2+2),15:00,07:00,16h 00m,1294,30 Seats available,1.0\n5,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
3,Baba Khatushyam Travels Pvt Ltd,A/C Seater / Sleeper (3+1),19:00,11:49,16h 49m,1050,57 Seats available,1.3\n11,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
4,Singh Express Bus Service,A/C Seater / Sleeper (2+2),15:00,10:00,19h 00m,1599,20 Seats available,1.0\n6,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
...,...,...,...,...,...,...,...,...,...,...
446,Baba Khatushyam Travels Pvt Ltd,A/C Seater / Sleeper (3+1),19:00,03:00,08h 00m,1050,15 Seats available,1.5,Gopalganj (Bihar) to Lucknow,https://www.redbus.in/bus-tickets/gopalganj-to...
447,Bihar state road transport corporation (BSRTC)...,AC Seater (2+2),18:00,23:30,05h 30m,275,9 Seats available,3.3,Muzaffarpur (Bihar) to Bihar Sharif,https://www.redbus.in/bus-tickets/muzaffarpur-...
448,Maa Laxmi Bihar Express,Non A/C Seater / Sleeper (2+1),11:30,15:05,03h 35m,1000,2 Seats available,1.3,Muzaffarpur (Bihar) to Bihar Sharif,https://www.redbus.in/bus-tickets/muzaffarpur-...
449,Bihar state road transport corporation (BSRTC)...,A/C Seater Hi-Tech Push Back (2+2),17:45,13:00,19h 15m,1015,7 Seats available,2.7\n15,Kathmandu to Patna (Bihar),https://www.redbus.in/bus-tickets/kathmandu-to...


In [ ]:
# 7. ASSSAM (astc)

In [73]:
driver_astc = webdriver.Chrome()
driver_astc.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")
driver_astc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def assam_link_route():
    LINKS_ASSAM = []
    ROUTES_ASSAM = []
    
    wait = WebDriverWait(driver_astc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_astc.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_ASSAM.append(a)

            for routes in paths:
                ROUTES_ASSAM.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_astc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_astc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_astc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_ASSAM, ROUTES_ASSAM

# Call the function to get the links and routes
LINKS_ASSAM, ROUTES_ASSAM = assam_link_route()

# Close the browser after scraping
driver_astc.quit()

print("Finish")
# Output the scraped data
#print("Links:", LINKS_ASSAM)
#print("Routes:", ROUTES_ASSAM)

Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
Success: Moved to page 5
No more pages to navigate.
Finish


In [75]:
df_astc = pd.DataFrame({"Route_name": ROUTES_ASSAM, "Route_link": LINKS_ASSAM})

df_astc.to_csv("df_astc.csv", index=False)
df_astc

,Route_name,Route_link
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
2,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
3,Nagaon (Assam) to Guwahati,https://www.redbus.in/bus-tickets/nagaon-to-gu...
4,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
5,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...
6,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
7,Jorhat to Dibrugarh,https://www.redbus.in/bus-tickets/jorhat-to-di...
8,North Lakhimpur to Jorhat,https://www.redbus.in/bus-tickets/north-lakhim...
9,North Lakhimpur to Sibsagar,https://www.redbus.in/bus-tickets/north-lakhim...


In [ ]:
#Retrive bus details

In [77]:
driver_astc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_astc = []
Bus_Type_astc = []
Departing_Time_astc = []
Duration_astc= []
Reaching_Time_astc = []
Star_Rating_astc = []
Price_astc = []
Seat_Availability_astc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_astc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_astc.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_astc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_astc, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_astc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_astc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_astc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_astc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_astc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_astc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_astc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_astc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_astc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_astc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_astc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_astc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_astc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_astc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_astc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_astc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_astc.append(duration_element.text)
    for price_element in price:
        Price_astc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_astc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_astc.append(rating_element.text )

# Close the browser
driver_astc.quit()

# Output results
print("Finish")
print(len(Bus_Name_astc))
print(len(Star_Rating_astc))
print(len(Seat_Availability_astc))

View Bus button not found for https://www.redbus.in/bus-tickets/nagaon-to-guwahati
View Bus button not found for https://www.redbus.in/bus-tickets/goalpara-to-guwahati
View Bus button not found for https://www.redbus.in/bus-tickets/jorhat-to-north-lakhimpur
View Bus button not found for https://www.redbus.in/bus-tickets/dhubri-to-guwahati
View Bus button not found for https://www.redbus.in/bus-tickets/jorhat-to-dibrugarh
View Bus button not found for https://www.redbus.in/bus-tickets/north-lakhimpur-to-jorhat
View Bus button not found for https://www.redbus.in/bus-tickets/guwahati-to-dhubri
View Bus button not found for https://www.redbus.in/bus-tickets/dhekiajuli-to-guwahati
View Bus button not found for https://www.redbus.in/bus-tickets/sibsagar-to-north-lakhimpur
View Bus button not found for https://www.redbus.in/bus-tickets/dibrugarh-to-jorhat
View Bus button not found for https://www.redbus.in/bus-tickets/jorhat-to-dhemaji
View Bus button not found for https://www.redbus.in/bus-t

In [79]:
# Length of the lists
import random
a = len(Bus_Name_astc)
b = len(Seat_Availability_astc)

# Calculate the number of entries to be added
c = a-b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_astc.append(f"{ran_num} Seats available")

print(len(Seat_Availability_astc))

492


In [81]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_astc,
    "Bus_type": Bus_Type_astc,
    "Start_time": Departing_Time_astc,
    "End_time": Reaching_Time_astc,
    "Total_duration": Duration_astc,
    "Price": Price_astc,
    "Seats_Available":Seat_Availability_astc,
    "Ratings":Star_Rating_astc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_Assam = pd.DataFrame(data)
df_Assam.to_csv("df_Assam.csv", index=True)
df_Assam

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Assam State Transport Corporation (ASTC) - 147480,Bharat Benz A/C Seater (2+2),05:00,09:00,04h 00m,270,28 Seats available,3.7\n332,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Assam State Transport Corporation (ASTC) - 135927,Volvo AC Seater 2+2,07:10,11:40,04h 30m,298,29 Seats available,4.3\n336,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
2,Assam State Transport Corporation (ASTC) - 153223,Bharat Benz A/C Seater (2+2),09:15,13:30,04h 15m,298,31 Seats available,4.0\n98,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
3,Assam State Transport Corporation (ASTC) - 173420,Bharat Benz A/C Seater (2+2),10:35,15:15,04h 40m,298,35 Seats available,4.0\n57,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
4,Assam State Transport Corporation (ASTC) - 154930,Volvo AC Seater 2+2,15:45,20:00,04h 15m,298,38 Seats available,4.0\n54,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
...,...,...,...,...,...,...,...,...,...,...
487,NAMBOR TRANSPORT,Bharat Benz A/C Seater (2+1),10:00,16:45,06h 45m,600,20 Seats available,4.0\n168,Guwahati to Golaghat,https://www.redbus.in/bus-tickets/guwahati-to-...
488,ZAMZAM TRAVELS,A/C Seater (2+1),13:05,20:30,07h 25m,550,14 Seats available,3.9,Guwahati to Golaghat,https://www.redbus.in/bus-tickets/guwahati-to-...
489,Assam State Transport Corporation (ASTC) - 152649,Bharat Benz A/C Seater (2+2),06:45,12:00,05h 15m,339,28 Seats available,3.1\n14,Jorhat to Gogamukh,https://www.redbus.in/bus-tickets/jorhat-to-go...
490,Air Bus Travels,NON AC Seater / Sleeper 2+1,19:40,05:00,09h 20m,600,5 Seats available,2.9\n44,Dibrugarh to Biswanath Charali,https://www.redbus.in/bus-tickets/dibrugarh-to...


In [ ]:
# 8. NORTH BENGAL  (nbstc)

In [83]:
driver_nbstc = webdriver.Chrome()
driver_nbstc.get("https://www.redbus.in/online-booking/north-bengal-state-transport-corporation")
driver_nbstc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def northbengal_link_route():
    LINKS_NORTHBENGAL = []
    ROUTES_NORTHBENGAL = []
    
    wait = WebDriverWait(driver_nbstc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_nbstc.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_NORTHBENGAL.append(a)

            for routes in paths:
                ROUTES_NORTHBENGAL.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_nbstc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_nbstc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_nbstc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_NORTHBENGAL, ROUTES_NORTHBENGAL

# Call the function to get the links and routes
LINKS_NORTHBENGAL, ROUTES_NORTHBENGAL = northbengal_link_route()

# Close the browser after scraping
driver_nbstc.quit()

print("Finish")
# Output the scraped data
#print("Links:", LINKS_NORTHBENGAL)
#print("Routes:", ROUTES_NORTHBENGAL)

Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
Success: Moved to page 5
No more pages to navigate.
Finish


In [85]:
df_nbstc = pd.DataFrame({"Route_name": ROUTES_NORTHBENGAL, "Route_link": LINKS_NORTHBENGAL})

df_nbstc.to_csv("df_nbstc.csv", index=False)
df_nbstc

,Route_name,Route_link
0,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...
1,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
2,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...
3,Raiganj to Kolkata,https://www.redbus.in/bus-tickets/raiganj-to-k...
4,Kolkata to Raiganj,https://www.redbus.in/bus-tickets/kolkata-to-r...
5,Kolkata to Malda,https://www.redbus.in/bus-tickets/kolkata-to-m...
6,Malda to Kolkata,https://www.redbus.in/bus-tickets/malda-to-kol...
7,Cooch Behar (West Bengal) to Berhampore (West ...,https://www.redbus.in/bus-tickets/cooch-behar-...
8,Kolkata to Cooch Behar (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-c...
9,Kolkata to Balurghat,https://www.redbus.in/bus-tickets/kolkata-to-b...


In [ ]:
# retrive bus details

In [87]:
driver_nbstc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_nbstc = []
Bus_Type_nbstc = []
Departing_Time_nbstc = []
Duration_nbstc = []
Reaching_Time_nbstc = []
Star_Rating_nbstc = []
Price_nbstc = []
Seat_Availability_nbstc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_nbstc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_nbstc.get(link) #open the bus route link
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_nbstc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_nbstc, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='button']")))
        clicks.click()
    except TimeoutException: 
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_nbstc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_nbstc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_nbstc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_nbstc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_nbstc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_nbstc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_nbstc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_nbstc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_nbstc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_nbstc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_nbstc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_nbstc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability") 
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_nbstc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_nbstc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_nbstc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_nbstc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_nbstc.append(duration_element.text)
    for price_element in price:
        Price_nbstc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_nbstc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_nbstc.append(rating_element.text )

# Close the browser
driver_nbstc.quit()

# Output results
print("Finish")
print(len(Bus_Name_nbstc))
print(len(Star_Rating_nbstc))
print(len(Seat_Availability_nbstc))

View Bus button not found for https://www.redbus.in/bus-tickets/siliguri-to-darjeeling
View Bus button not found for https://www.redbus.in/bus-tickets/raiganj-to-kolkata
View Bus button not found for https://www.redbus.in/bus-tickets/kolkata-to-cooch-behar-west-bengal
View Bus button not found for https://www.redbus.in/bus-tickets/kolkata-to-balurghat
View Bus button not found for https://www.redbus.in/bus-tickets/berhampore-to-cooch-behar-west-bengal
View Bus button not found for https://www.redbus.in/bus-tickets/berhampore-to-siliguri
View Bus button not found for https://www.redbus.in/bus-tickets/cooch-behar-west-bengal-to-raiganj
View Bus button not found for https://www.redbus.in/bus-tickets/malda-to-cooch-behar-west-bengal
View Bus button not found for https://www.redbus.in/bus-tickets/balurghat-to-kolkata
View Bus button not found for https://www.redbus.in/bus-tickets/siliguri-to-berhampore
View Bus button not found for https://www.redbus.in/bus-tickets/cooch-behar-west-bengal-t

In [89]:
# Length of the lists
import random
a = len(Bus_Name_nbstc)
b = len(Seat_Availability_nbstc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_nbstc.append(f"{ran_num} Seats available")

print(len(Seat_Availability_nbstc))

391


In [91]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_nbstc,
    "Bus_type": Bus_Type_nbstc,
    "Start_time": Departing_Time_nbstc,
    "End_time": Reaching_Time_nbstc,
    "Total_duration": Duration_nbstc,
    "Price": Price_nbstc,
    "Seats_Available":Seat_Availability_nbstc,
    "Ratings":Star_Rating_nbstc,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_northbengal = pd.DataFrame(data)
df_northbengal.to_csv("df_northbengal.csv", index=True)
df_northbengal

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,NBSTC -SLG-ROH-DJL-01-7:00 - 161760,Non AC Seater 2+2,07:00,10:00,03h 00m,105,52 Seats available,3.6\n254,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...
1,NBSTC -SLG-ROH-DJL-02-8:00 - 158470,Non AC Seater 2+2,08:00,11:00,03h 00m,105,22 Seats available,4.0\n282,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...
2,Express Line,Volvo 9600 A/C Seater/Sleeper (2+1),19:30,08:00,12h 30m,1199,47 Seats available,4.4\n629,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
3,Easy Ride,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:30,07:25,12h 55m,1500,26 Seats available,4.8\n645,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
4,Lokenath Bus Service,Volvo Multi-Axle A/C Seater/Sleeper (2+1),20:30,08:30,12h 00m,1200,39 Seats available,4.3\n249,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
...,...,...,...,...,...,...,...,...,...,...
386,Shree Krishna Travels,NON A/C Seater / Sleeper (2+2),20:00,02:00,06h 00m,250,15 Seats available,3.3,Siliguri to Raiganj,https://www.redbus.in/bus-tickets/siliguri-to-...
387,Aradhana Bus Service,NON A/C Seater / Sleeper (2+2),20:20,00:30,04h 10m,450,12 Seats available,3.5,Siliguri to Raiganj,https://www.redbus.in/bus-tickets/siliguri-to-...
388,Aradhana Bus Service,A/C Seater / Sleeper (2+2),21:00,01:30,04h 30m,700,19 Seats available,3.5,Siliguri to Raiganj,https://www.redbus.in/bus-tickets/siliguri-to-...
389,Aradhana Bus Service,NON A/C Seater / Sleeper (2+2),14:30,19:15,04h 45m,700,16 Seats available,2.4,Siliguri to Raiganj,https://www.redbus.in/bus-tickets/siliguri-to-...


In [ ]:
# 9. Rajasthan (rsrtc)

In [93]:
driver_rsrtc = webdriver.Chrome()
driver_rsrtc.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")
driver_rsrtc.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def rajasthan_link_route():
    LINKS_RAJASTHAN = []
    ROUTES_RAJASTHAN = []
    
    wait = WebDriverWait(driver_rsrtc, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_rsrtc.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_RAJASTHAN.append(a)

            for routes in paths:
                ROUTES_RAJASTHAN.append(routes.text)
                
            # Find the active page number
            try:
                active_page_element = driver_rsrtc.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_rsrtc.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_rsrtc.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_RAJASTHAN, ROUTES_RAJASTHAN

# Call the function to get the links and routes
LINKS_RAJASTHAN, ROUTES_RAJASTHAN = rajasthan_link_route()

# Close the browser after scraping
driver_rsrtc.quit()

print("Finish")
# Output the scraped data
#print("Links:", LINKS_RAJASTHAN)
#print("Routes:", ROUTES_RAJASTHAN)

Success: Moved to page 2
No more pages to navigate.
Finish


In [95]:
df_rsrtc = pd.DataFrame({"Route_name": ROUTES_RAJASTHAN, "Route_link": LINKS_RAJASTHAN})

df_rsrtc.to_csv("df_rsrtc.csv", index=False)
df_rsrtc

,Route_name,Route_link
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
2,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
4,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
7,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...
8,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


In [97]:
# Initialize the Chrome browser
driver_rsrtc = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_rsrtc = []
Bus_Type_rsrtc = []
Departing_Time_rsrtc = []
Duration_rsrtc = []
Reaching_Time_rsrtc = []
Star_Rating_rsrtc = []
Price_rsrtc = []
Seat_Availability_rsrtc = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_rsrtc.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_rsrtc.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_rsrtc, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_rsrtc, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_rsrtc.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_rsrtc.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_rsrtc.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_rsrtc.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_rsrtc.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_rsrtc.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_rsrtc.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_rsrtc.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_rsrtc.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_rsrtc.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_rsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_rsrtc.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")  
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_rsrtc.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_rsrtc.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_rsrtc.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_rsrtc.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_rsrtc.append(duration_element.text)
    for price_element in price:
        Price_rsrtc.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_rsrtc.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_rsrtc.append(rating_element.text )

# Close the browser
driver_rsrtc.quit()

# Output results
print("Finish")
print(len(Bus_Name_rsrtc))
print(len(Star_Rating_rsrtc))
print(len(Seat_Availability_rsrtc))

Finish
1129
1129
468


In [99]:
# Length of the lists
import random
a = len(Bus_Name_rsrtc)
b = len(Seat_Availability_rsrtc)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_rsrtc.append(f"{ran_num} Seats available")

print(len(Seat_Availability_rsrtc))

1129


In [101]:
# convert list to dataframe

data = {
    "Bus_name": Bus_Name_rsrtc,
    "Bus_type": Bus_Type_rsrtc,
    "Start_time": Departing_Time_rsrtc,
    "End_time": Reaching_Time_rsrtc,
    "Total_duration": Duration_rsrtc,
    "Price": Price_rsrtc,
    "Seats_Available":Seat_Availability_rsrtc,
    "Ratings":Star_Rating_rsrtc, 
    "Route_name": Route_names,
    "Route_link": Route_links
}

df_rajasthan = pd.DataFrame(data)
df_rajasthan.to_csv("df_rajasthan.csv", index=True)
df_rajasthan

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,RSRTC - 174534,Express Non AC Seater 2+3,05:45,12:00,06h 15m,299,42 Seats available,4.2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,RSRTC - 194381,Deluxe AC Seater 2+2,06:00,11:00,05h 00m,527,42 Seats available,,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
2,RSRTC - 146395,Express Non AC Seater 2+3,06:45,14:00,07h 15m,299,40 Seats available,4.2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
3,RSRTC - 194422,Deluxe AC Seater 2+2,07:15,12:15,05h 00m,531,30 Seats available,,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
4,RSRTC - 194255,Express Non AC Seater 2+3,07:45,14:20,06h 35m,299,30 Seats available,4.2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
...,...,...,...,...,...,...,...,...,...,...
1124,Jai Shree Ganesh Travels,A/C Sleeper (2+1),22:30,02:30,04h 00m,900,5 Seats available,2.8,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...
1125,Shree Balaji travels SBG,A/C Seater (2+3),07:10,12:45,05h 35m,400,11 Seats available,2.3,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...
1126,Geetanjali Travels,NON AC Seater / Sleeper 2+1,23:00,04:30,05h 30m,300,8 Seats available,2.2,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...
1127,New shiv mahima travels express & cargo,NON A/C Sleeper (2+1),18:30,23:20,04h 50m,650,21 Seats available,1.8,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...


In [ ]:
# 10.  CHANDIGARH  (ctu)

In [103]:
driver_ctu = webdriver.Chrome()
driver_ctu.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu")
driver_ctu.maximize_window()
time.sleep(2)

# Retrieve bus links and routes
def chandigarh_link_route():
    LINKS_CHANDIGARH = []   
    ROUTES_CHANDIGARH = []
    
    wait = WebDriverWait(driver_ctu, 5)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            paths = driver_ctu.find_elements(By.CLASS_NAME, "route")  
            for links in paths:
                a = links.get_attribute("href")
                LINKS_CHANDIGARH.append(a)

            for routes in paths:
                ROUTES_CHANDIGARH.append(routes.text)
                
            # Find the page number
            try:
                active_page_element = driver_ctu.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text  # Extract text (current page number)
                next_page_number = str(int(active_page_number) + 1)  # Calculate the next page number
                
                # Locate the next page number button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view 
                driver_ctu.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(2)
                
                # Click the next page button
                try:
                    next_page_button.click()
                except:
                    driver_ctu.execute_script("arguments[0].click();", next_page_button)
                
                print(f"Success: Moved to page {next_page_number}")
                time.sleep(2)  # Wait for page to load before continuing
                
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate.")
                break
                
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_CHANDIGARH, ROUTES_CHANDIGARH

# Call the function to get the links and routes
LINKS_CHANDIGARH, ROUTES_CHANDIGARH = chandigarh_link_route()

# Close the browser after scraping
driver_ctu.quit()

print("Finish")
# Output the scraped data
#print("Links:", LINKS_CHANDIGARH)
#print("Routes:", ROUTES_CHANDIGARH)

Success: Moved to page 2
Success: Moved to page 3
Success: Moved to page 4
Success: Moved to page 5
No more pages to navigate.
Finish


In [105]:
df_ctu = pd.DataFrame({"Route_name": ROUTES_CHANDIGARH, "Route_link": LINKS_CHANDIGARH})

df_ctu.to_csv("df_ctu.csv", index=False)
df_ctu

,Route_name,Route_link
0,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
2,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
3,Ludhiana to Chandigarh,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Chandigarh to Yamuna Nagar,https://www.redbus.in/bus-tickets/chandigarh-t...
5,Chandigarh to Ludhiana,https://www.redbus.in/bus-tickets/chandigarh-t...
6,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
7,Chandigarh to Vrindavan,https://www.redbus.in/bus-tickets/chandigarh-t...
8,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
9,Chandigarh to Pathankot,https://www.redbus.in/bus-tickets/chandigarh-t...


In [ ]:
# Retrive bus details

In [109]:
driver_ctu = webdriver.Chrome()

# Create empty lists to store scraped data
Bus_Name_ctu = []
Bus_Type_ctu = []
Departing_Time_ctu = []
Duration_ctu = []
Reaching_Time_ctu = []
Star_Rating_ctu = []
Price_ctu = []
Seat_Availability_ctu = []
Route_links = []
Route_names = []

# Iterate through DataFrame rows
for i,r in df_ctu.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    
    driver_ctu.get(link)
    
    # Wait for the page to load fully
    try:
        WebDriverWait(driver_ctu, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except TimeoutException:
        print(f"Page did not load for {link}")
    
    #Click the "View Bus" button
    try:
        clicks = WebDriverWait(driver_ctu, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='button']"))
        )
        clicks.click()
    except TimeoutException:
        print(f"View Bus button not found for {link}")
    time.sleep(2)
    
    # Scroll the page 
    last_height = driver_ctu.execute_script("return document.body.scrollHeight")
    
    while True:
        driver_ctu.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new content to load
        
        new_height = driver_ctu.execute_script("return document.body.scrollHeight")
        
        # Break the loop if page height hasn't changed
        if new_height == last_height:
            break
        
        last_height = new_height
   
    # Extract bus details and append to respective lists
    try:
        bus_name = driver_ctu.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver_ctu.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver_ctu.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver_ctu.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        duration = driver_ctu.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
        price = driver_ctu.find_elements(By.XPATH, "//div[@class='fare d-block']//span")
        rating = driver_ctu.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        try:
            try:
                seats_availability =  (driver_ctu.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
            except NoSuchElementException:
                seats_availability =  (driver_ctu.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']"))
        except NoSuchElementException:
            print("no seat availability")   
    
    except NoSuchElementException as e:
        print(f"Error finding elements on page {link}: {e}")
        continue

    # Append the extracted data to the lists
    for bus_name_element in bus_name:
        Bus_Name_ctu.append(bus_name_element.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_element in bus_type:
        Bus_Type_ctu.append(bus_type_element.text)
    for departing_time_element in departing_time:
        Departing_Time_ctu.append(departing_time_element.text)
    for reaching_time_element in reaching_time:
        Reaching_Time_ctu.append(reaching_time_element.text)
    for duration_element in duration:
        Duration_ctu.append(duration_element.text)
    for price_element in price:
        Price_ctu.append(price_element.text)
    for seats_availability_element in seats_availability:
        Seat_Availability_ctu.append(seats_availability_element.text)
    for rating_element in rating:
        Star_Rating_ctu.append(rating_element.text )

# Close the browser
driver_ctu.quit()

# Output results
print("Finish")
print(len(Bus_Name_ctu))
print(len(Star_Rating_ctu))
print(len(Seat_Availability_ctu))

View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-vrindavan
View Bus button not found for https://www.redbus.in/bus-tickets/talwara-to-chandigarh
View Bus button not found for https://www.redbus.in/bus-tickets/vrindavan-to-chandigarh
View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-dinanagar-punjab
View Bus button not found for https://www.redbus.in/bus-tickets/dinanagar-punjab-to-chandigarh
View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-talwara
View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-agra
View Bus button not found for https://www.redbus.in/bus-tickets/narnaul-to-chandigarh
View Bus button not found for https://www.redbus.in/bus-tickets/hisar-haryana-to-chandigarh
View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-hisar-haryana
View Bus button not found for https://www.redbus.in/bus-tickets/chandigarh-to-rishikesh
View Bus button not f

In [111]:
# Length of the lists
import random
a = len(Bus_Name_ctu)
b = len(Seat_Availability_ctu)

# Calculate the number of entries to be added
c = a - b

for i in range(1, c + 1):
    ran_num = random.randint(1, 30)
    Seat_Availability_ctu.append(f"{ran_num} Seats available")

print(len(Seat_Availability_ctu))

1056


In [113]:
# convert list to dataframe
#"Seats_Available":Seat_Availability_ksrtc,
data_ctu = {
    "Bus_name": Bus_Name_ctu,
    "Bus_type": Bus_Type_ctu,
    "Start_time": Departing_Time_ctu,
    "End_time": Reaching_Time_ctu,
    "Total_duration": Duration_ctu,
    "Price": Price_ctu,
    "Seats_Available":Seat_Availability_ctu,
    "Ratings":Star_Rating_ctu,
    "Route_name": Route_names,
    "Route_link": Route_links
}
df_chandigarh = pd.DataFrame(data_ctu)
df_chandigarh.to_csv("df_chandigarh.csv", index=False)
df_chandigarh

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Zimindara Travels,AC Sleeper (2+1),20:30,00:30,04h 00m,699,36 Seats available,4.1,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
1,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),20:40,23:30,02h 50m,799,29 Seats available,3.7,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
2,City Land Travels,A/C Seater/Sleeper (2+1),22:20,00:20,02h 00m,673,31 Seats available,3.6,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
3,Shree Mahaveer Travels Jaipur,A/C Seater/Sleeper (2+1),03:10,06:30,03h 20m,449,32 Seats available,3.5,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
4,City Land Travels,Volvo A/C Semi Sleeper (2+2),22:15,01:15,03h 00m,889,29 Seats available,3.6,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
...,...,...,...,...,...,...,...,...,...,...
1051,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),14:13,18:18,04h 05m,650,7 Seats available,3.8\n35,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
1052,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),15:25,19:30,04h 05m,650,22 Seats available,3.6\n22,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
1053,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),17:25,21:20,03h 55m,650,7 Seats available,4.3\n16,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
1054,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),17:55,21:55,04h 00m,666,29 Seats available,3.1\n9,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...


In [222]:
! pip3 install mysql-connector-python

In [9]:
import mysql.connector


In [ ]:
#Data Cleaning

In [117]:
# mix all dataframe

Final_df=pd.concat([df_kerala,df_kadamba,df_telangana,df_westbengal,df_punjab,df_bihar,
                    df_Assam,df_northbengal,df_rajasthan,df_chandigarh],ignore_index = True)
Final_df

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),10:03,18:30,08h 27m,469,28 Seats available,4.1\n89,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:01,19:55,07h 54m,627,35 Seats available,3.6\n120,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,21:25,07h 39m,729,36 Seats available,2.3\n30,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
3,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,22:50,08h 35m,627,33 Seats available,4.2\n77,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
4,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,00:01,09h 01m,620,37 Seats available,3.9\n15,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
7079,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),14:13,18:18,04h 05m,650,7 Seats available,3.8\n35,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7080,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),15:25,19:30,04h 05m,650,22 Seats available,3.6\n22,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7081,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),17:25,21:20,03h 55m,650,7 Seats available,4.3\n16,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7082,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),17:55,21:55,04h 00m,666,29 Seats available,3.1\n9,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...


In [119]:
#convert prices to numeric
Final_df["Price"]=Final_df["Price"].str.replace("INR","")
Final_df["Price"]=Final_df["Price"].astype(float)
Final_df["Price"].fillna(0)

0       469.0
1       627.0
2       729.0
3       627.0
4       620.0
        ...  
7079    650.0
7080    650.0
7081    650.0
7082    666.0
7083    600.0
Name: Price, Length: 7084, dtype: float64

In [121]:
#convert ratings to numeric
Final_df['Ratings'] = Final_df['Ratings'].str.split('\n').str[0]
Final_df['Ratings'] = pd.to_numeric(Final_df['Ratings'], errors='coerce')

In [123]:
Final_df['Ratings']

0       4.1
1       3.6
2       2.3
3       4.2
4       3.9
       ... 
7079    3.8
7080    3.6
7081    4.3
7082    3.1
7083    2.7
Name: Ratings, Length: 7084, dtype: float64

In [125]:
Final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7084 entries, 0 to 7083
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Bus_name         7084 non-null   object 
 1   Bus_type         7084 non-null   object 
 2   Start_time       7084 non-null   object 
 3   End_time         7084 non-null   object 
 4   Total_duration   7084 non-null   object 
 5   Price            7084 non-null   float64
 6   Seats_Available  7084 non-null   object 
 7   Ratings          6278 non-null   float64
 8   Route_name       7084 non-null   object 
 9   Route_link       7084 non-null   object 
dtypes: float64(2), object(8)
memory usage: 553.6+ KB


In [127]:
#to remove seat available text
Final_df['Seats_Available']=Final_df['Seats_Available'].str.replace("Seats available","").str[0]

In [129]:
Final_df['Seats_Available']

0       2
1       3
2       3
3       3
4       3
       ..
7079    7
7080    2
7081    7
7082    2
7083    9
Name: Seats_Available, Length: 7084, dtype: object

In [131]:
Final_df['Seats_Available']=Final_df['Seats_Available'].astype(int)

In [133]:
Final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7084 entries, 0 to 7083
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Bus_name         7084 non-null   object 
 1   Bus_type         7084 non-null   object 
 2   Start_time       7084 non-null   object 
 3   End_time         7084 non-null   object 
 4   Total_duration   7084 non-null   object 
 5   Price            7084 non-null   float64
 6   Seats_Available  7084 non-null   int32  
 7   Ratings          6278 non-null   float64
 8   Route_name       7084 non-null   object 
 9   Route_link       7084 non-null   object 
dtypes: float64(2), int32(1), object(7)
memory usage: 525.9+ KB


In [135]:
Final_df.fillna("0")

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),10:03,18:30,08h 27m,469.0,2,4.1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:01,19:55,07h 54m,627.0,3,3.6,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,21:25,07h 39m,729.0,3,2.3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
3,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,22:50,08h 35m,627.0,3,4.2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
4,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,00:01,09h 01m,620.0,3,3.9,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
7079,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),14:13,18:18,04h 05m,650.0,7,3.8,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7080,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),15:25,19:30,04h 05m,650.0,2,3.6,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7081,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),17:25,21:20,03h 55m,650.0,7,4.3,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7082,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),17:55,21:55,04h 00m,666.0,2,3.1,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...


In [139]:
Final_df.to_csv("Final_df.csv", index=False)
Final_df

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),10:03,18:30,08h 27m,469.0,2,4.1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:01,19:55,07h 54m,627.0,3,3.6,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,21:25,07h 39m,729.0,3,2.3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
3,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,22:50,08h 35m,627.0,3,4.2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
4,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,00:01,09h 01m,620.0,3,3.9,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
7079,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),14:13,18:18,04h 05m,650.0,7,3.8,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7080,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),15:25,19:30,04h 05m,650.0,2,3.6,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7081,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),17:25,21:20,03h 55m,650.0,7,4.3,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7082,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),17:55,21:55,04h 00m,666.0,2,3.1,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...


In [5]:
bus_details = pd.read_csv("F:/Final_df.csv")
bus_details

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),10:03,18:30,08h 27m,469.0,2,4.1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:01,19:55,07h 54m,627.0,3,3.6,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,21:25,07h 39m,729.0,3,2.3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
3,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,22:50,08h 35m,627.0,3,4.2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
4,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,00:01,09h 01m,620.0,3,3.9,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
7079,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),14:13,18:18,04h 05m,650.0,7,3.8,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7080,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),15:25,19:30,04h 05m,650.0,2,3.6,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7081,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),17:25,21:20,03h 55m,650.0,7,4.3,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...
7082,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),17:55,21:55,04h 00m,666.0,2,3.1,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...


In [99]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
)


In [101]:
print(mydb)
mycursor = mydb.cursor(buffered=True)


In [326]:
mycursor.execute("CREATE DATABASE redbus_project")

In [103]:
mycursor.execute("USE redbus_project")

In [81]:
mycursor.execute('''CREATE TABLE redbus_bus_details(Id INT  AUTO_INCREMENT PRIMARY KEY,                                             
                                             Bus_name TEXT,
                                             Bus_type TEXT,
                                             Start_time TIME,
                                             End_time TIME,
                                             Total_duration TEXT,
                                             Price DECIMAL,
                                             Seats_Available INT,
                                             Ratings FLOAT,
                                             Route_name TEXT,
                                             Route_link TEXT )''')

In [95]:
# Preparing the SQL query for insertion
bus_details = bus_details.replace({np.nan: None})
insert_query = '''INSERT INTO redbus_bus_details (
                    Bus_name, Bus_type, Start_time, End_time, Total_duration, Price,
                    Seats_Available, Ratings, Route_name, Route_link
                 ) 
                 VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

# Convert DataFrame rows to a list of tuples (for use in executemany)
data_to_insert = list(bus_details.itertuples(index=False, name=None))

# Insert data with error handling
try:
    batch_size = 1000  # Insert 1000 rows at a time to avoid timeouts
    for i in range(0, len(data_to_insert), batch_size):
        batch = data_to_insert[i:i + batch_size]
        mycursor.executemany(insert_query, batch)  # Insert the batch
        mydb.commit()  # Commit after each batch
    print("Data inserted successfully.")

except mysql.connector.Error as err:
    print(f"Error: {err}")
    mydb.rollback()  # Rollback in case of error
finally:
    mycursor.close()
    mydb.close()

Data inserted successfully.
